# Install required libraries

In [ ]:
!pip install telethon python-telegram-bot apscheduler nest_asyncio python-dotenv httpx

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.0/722.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.3/702.3 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 3.1 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26347 sha256=b1fcc9e191355d7d5e42c9d6e7494936b6a40637e28a25e183920d3b2c495c3b
  Stored in directory: /root/.cache/pip/wheels/4e/52/33/010d0843550bffb6a591b11629070ae140c0ad4f53e68a3bd3
Successfully built pyaes


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import re
import os
import sqlite3
import asyncio
import httpx
from telethon import TelegramClient, events
from telegram import Update
from telegram.ext import Application, CommandHandler, ContextTypes
from apscheduler.schedulers.asyncio import AsyncIOScheduler
from datetime import datetime
from telethon import TelegramClient
from telegram.ext import Application, CommandHandler


#Google Drive for persistent storage
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    pass

# Create directories
os.makedirs("/content/drive/MyDrive/ai_news_bot/sessions", exist_ok=True)
os.makedirs("/content/drive/MyDrive/ai_news_bot/database", exist_ok=True)


Mounted at /content/drive



##Config Section

In [ ]:

class Config:
    def __init__(self):
        self.API_ID = os.getenv('TELEGRAM_API_ID', '23135262')  # Replace with your own  TELEGRAM_API_ID
        self.API_HASH = os.getenv('TELEGRAM_API_HASH', '7533fd16a5abfd1f8f8dda1733e92ee4')  # Replace TELEGRAM_API_HASH
        self.BOT_TOKEN = os.getenv('TELEGRAM_BOT_TOKEN', '7691879278:AAGhezcgkQPt0zhJP6GINFAGe7o9xcZ1hwY')  #Replace your own TELEGRAM_BOT_TOKEN
        self.ADMINS = [int(admin) for admin in os.getenv('ADMINS', ' ### ').split(',') if admin]  # please Replace your own ID Account Telegram if you want be a ADMIN into this BOT
        self.DB_PATH = '/content/drive/MyDrive/ai_news_bot/database/news_bot.db'
        self.SESSION_PATH = '/content/drive/MyDrive/ai_news_bot/sessions/news_bot_session'
        self.FETCHER_SESSION_PATH = '/content/drive/MyDrive/ai_news_bot/sessions/news_bot_fetcher_session'
        self.AI_KEYWORDS = ['Artificial Intelligence', 'AI', 'ChatGPT', 'GPT', 'LLM', 'Machine Learning', 'Deep Learning']
        self.DEFAULT_SOURCES = [
            ("AI Neural", "https://t.me/artificial_intelligence_neural"),
            ("Agineer AI", "https://t.me/agineer_ai"),
            ("GPT Anthropic", "https://t.me/gpt_anthropic"),
            ("AI Post", "https://t.me/aipost"),
            ("AI Posted", "https://t.me/aiposted"),
            ("NEWS AI", "https://t.me/news_aii"),
            ("AI News", "https://t.me/ai_news2"),
            ("Choise.ai News", "https://t.me/choise_com"),
            ("BrainAid AI News", "https://t.me/BrainNeuroassistant"),
            ("AI News | Midjourney & ChatGPT", "https://t.me/Ai_Newsss"),
            ("NOTMEME Agent News", "https://t.me/notmeme_agent_ai"),
            ("PROAI", "https://t.me/pro_ai_news"),
            ("AI_NEWS_FOR_TESTING_BOT_TEL", "https://t.me/AI_NEWS_FOR_TESTING_BOT_TEL"),
            ("AINEERING", "https://t.me/aineeringdotcom")
        ]

config = Config()



##Database Class

In [ ]:

class Database:
    def __init__(self):
        try:
            self.conn = sqlite3.connect(config.DB_PATH, check_same_thread=False) #Connect to DB
            self.create_tables() #Create tables if needed
            self.init_default_sources()
        except sqlite3.OperationalError as e:
            print(f"Database connection failed: {e}")
            raise

    def create_tables(self):
        cursor = self.conn.cursor()
        cursor.executescript('''
        CREATE TABLE IF NOT EXISTS sources (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            url TEXT NOT NULL UNIQUE,
            channel_id TEXT,
            is_active BOOLEAN DEFAULT 1,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        CREATE TABLE IF NOT EXISTS users (
            chat_id INTEGER PRIMARY KEY,
            username TEXT,
            first_name TEXT,
            last_name TEXT,
            is_admin BOOLEAN DEFAULT 0,
            subscribed BOOLEAN DEFAULT 1,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        CREATE TABLE IF NOT EXISTS news (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            source_id INTEGER,
            content TEXT,
            links TEXT,
            date TIMESTAMP,
            message_id INTEGER,
            channel_id INTEGER,
            FOREIGN KEY (source_id) REFERENCES sources (id)
        );
        CREATE TABLE IF NOT EXISTS sent_news (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            news_id INTEGER,
            user_id INTEGER,
            sent_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            UNIQUE(news_id, user_id)
        );
        ''')
        self.conn.commit()

    def init_default_sources(self):
        for name, url in config.DEFAULT_SOURCES:
            try:
                self.conn.execute("INSERT OR IGNORE INTO sources (name, url) VALUES (?, ?)", (name, url))
                self.conn.commit()
            except Exception as e:
                print(f"Error adding default source {name}: {e}")

    def add_source(self, name, url):
        try:
            self.conn.execute("INSERT INTO sources (name, url) VALUES (?, ?)", (name, url))
            self.conn.commit()
            return True
        except sqlite3.IntegrityError:
            return False

    def update_channel_id(self, url, channel_id):
        try:
            self.conn.execute("UPDATE sources SET channel_id = ? WHERE url = ?", (channel_id, url))
            self.conn.commit()
            return True
        except Exception as e:
            print(f"Error updating channel ID: {e}")
            return False

    def get_sources(self):
        cursor = self.conn.cursor()
        cursor.execute("SELECT id, name, url, channel_id FROM sources WHERE is_active = 1")
        return cursor.fetchall()

    def get_source_by_channel_id(self, channel_id):
        cursor = self.conn.cursor()
        cursor.execute("SELECT id, name, url FROM sources WHERE channel_id = ?", (channel_id,))
        return cursor.fetchone()

    def add_user(self, user):
        try:
            self.conn.execute(
                '''INSERT INTO users
                (chat_id, username, first_name, last_name, is_admin)
                VALUES (?, ?, ?, ?, ?)
                ON CONFLICT(chat_id) DO UPDATE SET
                username=excluded.username,
                first_name=excluded.first_name,
                last_name=excluded.last_name,
                is_admin=?''',
                (user.id, user.username, user.first_name, user.last_name, user.id in config.ADMINS, user.id in config.ADMINS)
            )
            self.conn.commit()
            return True
        except Exception as e:
            print(f"Error adding user: {e}")
            return False

    def get_subscribed_users(self):
        cursor = self.conn.cursor()
        cursor.execute("SELECT chat_id FROM users WHERE subscribed = 1")
        return [row[0] for row in cursor.fetchall()]

    def add_news(self, source_id, content, links, date, message_id, channel_id):
        try:
            cursor = self.conn.cursor()
            cursor.execute(
                '''INSERT INTO news
                (source_id, content, links, date, message_id, channel_id)
                VALUES (?, ?, ?, ?, ?, ?)''',
                (source_id, content, ','.join(links), date, message_id, channel_id)
            )
            self.conn.commit()
            return cursor.lastrowid  #Return ID of the new row
        except Exception as e:
            print(f"Error adding news: {e}")
            return None

    def news_exists(self, message_id, channel_id):
        cursor = self.conn.cursor()
        cursor.execute('SELECT 1 FROM news WHERE message_id=? AND channel_id=?', (message_id, channel_id))
        return cursor.fetchone() is not None

    def mark_news_sent(self, news_id, user_id):
        try:
            self.conn.execute('INSERT OR IGNORE INTO sent_news (news_id, user_id) VALUES (?, ?)', (news_id, user_id))
            self.conn.commit()
            return True
        except Exception as e:
            print(f"Error marking news as sent: {e}")
            return False

    def is_admin(self, chat_id):
        cursor = self.conn.cursor()
        cursor.execute("SELECT is_admin FROM users WHERE chat_id = ?", (chat_id,))
        result = cursor.fetchone()
        return result[0] if result else False

    def close(self):
        self.conn.close()



##Real-Time News Listener

In [ ]:
class RealTimeNewsListener:

    def __init__(self, db, bot_app):  # bot_app is Application object
        self.db = db
        self.bot = bot_app.bot  #   Accessing the bot via the Application
        self.client = TelegramClient(config.FETCHER_SESSION_PATH, config.API_ID, config.API_HASH)
        self.running = True

    async def start(self):
        await self.client.start()
        print(" listener started...")
        await self.initialize_channels()
        self.client.add_event_handler(self.handle_new_message, events.NewMessage)
        await self.client.run_until_disconnected()

    async def initialize_channels(self):
        sources = self.db.get_sources()
        for source_id, name, url, channel_id in sources:
            if not channel_id:
                try:
                    entity = await self.client.get_entity(url.replace('https://t.me/',  '@'))
                    self.db.update_channel_id(url, entity.id)
                    print(f"Registered channel: {name} (ID: {entity.id})")
                except Exception as e:
                    print(f"Error initializing channel {name}: {e}")

    async def handle_new_message(self, event):
        try:
            message = event.message
            if not message.text:
                return

            channel_id = message.chat_id
            source_info = self.db.get_source_by_channel_id(channel_id)
            if not source_info:
                return

            source_id, source_name, source_url = source_info

            if not self.is_ai_related(message.text): #Check if message is AI-related
                return

            if self.db.news_exists(message.id, channel_id): #Check for duplicates
                return

            links = re.findall(r'https?://\S+|www\.\S+', message.text) #Extract URLs from message
            news_id = self.db.add_news(
                source_id, message.text, links, message.date, message.id, channel_id
            ) #Add news to DB
            if not news_id:
                return

            formatted_news = self.format_news_item(source_name, source_url, {
                'text': message.text, 'date': message.date, 'links': links
            }) #Format news
            await self.send_to_subscribers(formatted_news, news_id)

        except Exception as e:
            print(f"Error handling new message: {e}")

    async def send_to_subscribers(self, message, news_id):
        users = self.db.get_subscribed_users()
        for user_id in users:
            try:
                await self.bot.send_message(
                    chat_id=user_id,
                    text=message,
                    parse_mode="HTML",
                    disable_web_page_preview=True
                )
                self.db.mark_news_sent(news_id, user_id)
                await asyncio.sleep(0.5)  # Avoid Telegram rate limits
            except Exception as e:
                print(f"Error sending to user {user_id}: {e}")

    def is_ai_related(self, text):
        if not text:
            return False
        text_lower = text.lower()
        return any(re.search(rf'\b{re.escape(keyword.lower())}\b', text_lower) for keyword in config.AI_KEYWORDS)

    def format_news_item(self, source_name, source_url, news_item):
        links = '\n'.join(news_item['links']) if news_item['links'] else "No links"
        return f"""
        <b>{source_name}</b>
          {news_item['date'].strftime('%Y-%m-%d %H:%M')}
          {source_url}
        {news_item['text'][:500]}{'...' if len(news_item['text']) > 500 else ''}
         Links:
        {links}
        """


In [ ]:
class TelegramNewsFetcher:
    #Fetches news from Telegram channels.
    def __init__(self):
      #Initializes the Telegram client.
        self.client = TelegramClient(config.FETCHER_SESSION_PATH, config.API_ID, config.API_HASH)

    async def fetch_channel_messages(self, channel_url, limit=5):
        try:
            async with self.client:
                messages = await self.client.get_messages(channel_url, limit=limit)
                return [
                    {
                        'text': msg.text,
                        'date': msg.date,
                        'id': msg.id,
                        'links': self.extract_links(msg.text) if msg.text else [],
                        'channel_id': msg.chat_id
                    }
                    for msg in messages if msg.text
                ]
        except Exception as e:
            print(f"Error fetching messages from {channel_url}: {str(e)}")
            return [] #Return empty list on error

    @staticmethod
    def extract_links(text):
        url_pattern = re.compile(r'https?://\S+|www\.\S+')
        return url_pattern.findall(text)

    async def close(self):
        await self.client.disconnect()

## Main Bot Part

In [ ]:
class AINewsBot:
    def __init__(self):
        self.db = Database()  # Database connection
        self.fetcher = TelegramNewsFetcher()
        self.scheduler = AsyncIOScheduler() # Scheduler for daily news

        self.application = Application.builder().token(config.BOT_TOKEN).build() # Telegram app
        self.realtime_listener = RealTimeNewsListener(self.db, self.application) # Real time listener
        self.setup_handlers()  # Register command handlers
        self.setup_scheduler() # Set up daily news scheduler
        self.setup_realtime_listener()


    def setup_handlers(self):
        handlers = [
            CommandHandler("start", self.start),
            CommandHandler("help", self.help),
            CommandHandler("subscribe", self.subscribe),
            CommandHandler("unsubscribe", self.unsubscribe),
            CommandHandler("news", self.get_latest_news),
            CommandHandler("list_sources", self.list_sources),
            CommandHandler("add_source", self.add_source),
            CommandHandler("send_daily", self.send_daily_news),
            CommandHandler("stats", self.get_stats),

        ]
        for handler in handlers:
            self.application.add_handler(handler)

    def setup_scheduler(self):
        # Daily news job
        self.scheduler.add_job(self.send_daily_news_job, 'cron', hour=8)
        self.scheduler.start()

    def setup_realtime_listener(self):
        """Start listener in background using the global event loop"""
        loop = asyncio.get_event_loop()  #   Correct way to get loop
        loop.create_task(self.realtime_listener.start())

    async def start(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Start command and bot guide"""
        user = update.effective_user
        self.db.add_user(user)
        welcome_msg = """
          Welcome to the AI News Reader Bot!
          Get AI news from top sources

          User Commands:
          /start - Show this guide
          /subscribe - Subscribe to daily news
          /unsubscribe - Unsubscribe
          /news - Get latest news now
          /list_sources - Show news sources

          Admin Commands:
          /add_source [name] [url] - Add news source
          /send_daily - Send daily news now
          /stats - Show bot statistics

          """
        await update.message.reply_text(welcome_msg)

    async def help(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Show command help"""
        await self.start(update, context)

    async def subscribe(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Subscribe to newsletter"""
        user = update.effective_user
        self.db.conn.execute("UPDATE users SET subscribed = 1 WHERE chat_id = ?", (user.id,))
        self.db.conn.commit()
        await update.message.reply_text("  You've successfully subscribed to the AI news feed!")

    async def unsubscribe(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Unsubscribe from newsletter"""
        user = update.effective_user
        self.db.conn.execute("UPDATE users SET subscribed = 0 WHERE chat_id = ?", (user.id,))
        self.db.conn.commit()
        await update.message.reply_text("  You've unsubscribed from the AI news feed.")

    async def add_source(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Add new news source (admin only)"""
        if not self.db.is_admin(update.effective_user.id):
            await update.message.reply_text("  This command is only available for admins.")
            return

        args = context.args
        if len(args) < 2:
            await update.message.reply_text("  Please enter channel name and URL.\nExample:\n/add_source channel_name https://t.me/channel_url")
            return

        name = ' '.join(args[:-1])
        url = args[-1]

        if not url.startswith('https://t.me/'):
            await update.message.reply_text("  Channel URL must start with https://t.me/")
            return

        if self.db.add_source(name, url):
            await update.message.reply_text(f"  Channel '{name}' added successfully!")
        else:
            await update.message.reply_text("  This channel has already been added")

    async def list_sources(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """List available news sources"""
        sources = self.db.get_sources()
        if not sources:
            await update.message.reply_text("  No channels have been registered yet")
            return

        message = "  Registered AI News Channels:\n\n"
        for idx, (_, name, url, _) in enumerate(sources, start=1):
            message += f"{idx}. {name}\n  {url}\n\n"

        await update.message.reply_text(message)

    async def get_latest_news(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Get latest news from various sources"""
        user_id = update.effective_user.id
        sources = self.db.get_sources()
        if not sources:
            await context.bot.send_message(chat_id=user_id, text="  No channels have been registered yet")
            return

        await context.bot.send_message(chat_id=user_id, text=f"  Fetching latest news from {len(sources)} channels...")

        for source_id, name, url, _ in sources:
            try:
                messages = await self.fetcher.fetch_channel_messages(url)
                ai_news = [msg for msg in messages if self.is_ai_related(msg['text'])]

                if ai_news:
                    for news in ai_news[:3]:  # Max 3 news items per channel
                        formatted = self.format_news_item(name, url, news)
                        await context.bot.send_message(
                            chat_id=user_id,
                            text=formatted,
                            parse_mode="HTML",
                            disable_web_page_preview=True
                        )
                await asyncio.sleep(2)  # Delay to avoid Telegram limits

            except Exception as e:
                await context.bot.send_message(chat_id=user_id, text=f"  Error fetching from {name}:\n{str(e)}")

    async def send_daily_news(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Send daily news command (admin only)"""
        if not self.db.is_admin(update.effective_user.id):
            await update.message.reply_text("  This command is only available for admins.")
            return

        await update.message.reply_text("  Preparing and sending daily news...")
        await self.send_news_to_all()
        await update.message.reply_text("  Daily news sent successfully!")

    async def send_daily_news_job(self):
        """Scheduled job for daily news delivery"""
        await self.send_news_to_all()

    async def send_news_to_all(self):
        """Send news to all subscribed users"""
        sources = self.db.get_sources()
        if not sources:
            return

        for source_id, name, url, _ in sources:
            try:
                messages = await self.fetcher.fetch_channel_messages(url)
                ai_news = [msg for msg in messages if self.is_ai_related(msg['text'])]

                for news in ai_news[:5]:  # Max 5 news items per source
                    formatted = self.format_news_item(name, url, news)
                    await self.broadcast_to_subscribers(formatted)
                await asyncio.sleep(2)

            except Exception as e:
                print(f"Error sending daily news from {name}: {str(e)}")

    async def broadcast_to_subscribers(self, message):
        """Send message to all subscribed users"""
        users = self.db.get_subscribed_users()
        for user_id in users:
            try:
                await self.application.bot.send_message(
                    chat_id=user_id,
                    text=message,
                    parse_mode="HTML",
                    disable_web_page_preview=True
                )
            except Exception as e:
                print(f"Error sending to user {user_id}: {str(e)}")
            await asyncio.sleep(0.5)  # Rate limiting

    async def get_stats(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Get statistics (admin only)"""
        if not self.db.is_admin(update.effective_user.id):
            await update.message.reply_text(" This command is only available for admins.")
            return

        cursor = self.db.conn.cursor()
        #Total users
        cursor.execute("SELECT COUNT(*) FROM users")
        total_users = cursor.fetchone()[0]
        # Active users
        cursor.execute("SELECT COUNT(*) FROM users WHERE subscribed = 1")
        active_users = cursor.fetchone()[0]
        # News sources
        cursor.execute("SELECT COUNT(*) FROM sources WHERE is_active = 1")
        active_sources = cursor.fetchone()[0]
        # News items
        cursor.execute("SELECT COUNT(*) FROM news")
        total_news = cursor.fetchone()[0]

        stats_msg = f"""
                      Bot Statistics:
                      Total users: {total_users}
                      Active subscribers: {active_users}
                      Active news sources: {active_sources}
                      Total news processed: {total_news}
                    """
        await update.message.reply_text(stats_msg)


    def is_ai_related(self, text):
        """Check if text is AI-related"""
        if not text:
            return False
        text_lower = text.lower()
        return any(keyword.lower() in text_lower for keyword in config.AI_KEYWORDS)

    def format_news_item(self, source_name, source_url, news_item):
        """Format news item for display"""
        links = '\n'.join(news_item['links']) if news_item['links'] else "No links"
        return f"""
<b>{source_name}</b>
  {news_item['date'].strftime('%Y-%m-%d %H:%M')}
  {source_url}
 {news_item['text'][:300]}...
  Links:
{links}
"""


    def setup_listener(self): #Redundant - remove
        loop = self.application._loop
        loop.create_task(self.realtime_listener.start())


    def run(self):
        """Main bot execution"""
        print(" AI News Reader Bot is ready")
        self.application.run_polling()


##Run Bot

In [ ]:
# Start the bot
if __name__ == "__main__":
    bot = AINewsBot()  # Initialize
    bot.run()  # Go

 AI News Reader Bot is ready
 listener started...
